<a href="https://colab.research.google.com/github/ashishpal2702/HumanActivityrecognition/blob/main/Logistic_Regression_and_Classification_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

We will be using the [Human Activity Recognition with Smartphones](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) database, which was built from the recordings of study participants performing activities of daily living (ADL) while carrying a smartphone with an embedded inertial sensors. The objective is to classify activities into one of the six activities (walking, walking upstairs, walking downstairs, sitting, standing, and laying) performed.

For each record in the dataset it is provided: 

- Triaxial acceleration from the accelerometer (total acceleration) and the estimated body acceleration. 
- Triaxial Angular velocity from the gyroscope. 
- A 561-feature vector with time and frequency domain variables. 
- Its activity label. 

More information about the features is available on the website above.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
from __future__ import print_function
import os
data_path = [ 'data']

from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import ExtraTreesClassifier


In [2]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

## 1. Data Import

Import the data and do the following:

* Examine the data types--there are many columns, so it might be wise to use value counts
* Determine if the floating point values need to be scaled
* Determine the breakdown of each activity
* Encode the activity label as an integer

In [3]:
import pandas as pd
import numpy as np
import os
#filepath = '/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data//Human_Activity_Recognition_Using_Smartphones_Data_augmented_data.gzip'
filepath = '/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/Human_Activity_Recognition_Using_Smartphones_Data_augmented_timedata_sample.gzip'
data = pd.read_parquet(filepath)

In [4]:
data.shape

(100000, 563)

In [6]:
data.head()


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,date_time
156800,0.262871,0.013216,-0.086324,0.200015,-0.156414,-0.278659,0.111663,-0.230944,-0.271890,0.580217,...,-0.741163,0.121548,-0.833951,-0.904012,0.643397,-0.994826,0.094677,0.023043,WALKING_DOWNSTAIRS,2020-01-01 01:00:00
105861,0.279888,-0.016259,-0.108582,-0.956027,-0.967547,-0.972543,-0.955547,-0.963617,-0.969774,-0.907162,...,-0.733630,-0.110451,0.055678,0.006752,-0.149780,0.562788,-0.284655,-0.730306,LAYING,2020-01-01 01:01:00
14961,0.057557,-0.065914,-0.318505,-0.204595,-0.313841,0.121934,-0.244170,-0.336566,0.132129,-0.042670,...,-0.400585,0.077258,0.625433,0.948960,-0.535123,-0.556217,0.199724,0.317411,WALKING_UPSTAIRS,2020-01-01 01:02:00
182560,0.166638,0.009589,-0.093877,0.075222,0.129887,-0.170282,-0.037054,0.049850,-0.245201,0.404261,...,-0.477741,0.685372,0.344247,0.781544,0.139801,-0.855667,0.187082,0.038144,WALKING_DOWNSTAIRS,2020-01-01 01:03:00
156371,0.277392,-0.013888,-0.114378,-0.996323,-0.993373,-0.989750,-0.997365,-0.993227,-0.989991,-0.939918,...,-0.986440,0.043316,-0.202663,-0.192810,-0.372000,-0.673985,-0.055019,-0.155311,SITTING,2020-01-01 01:04:00


In [7]:
sensors = set()
for col in data.columns:
    sensors.add(col.split("-")[0])

In [8]:
sensors

{'Activity',
 'angle(X,gravityMean)',
 'angle(Y,gravityMean)',
 'angle(Z,gravityMean)',
 'angle(tBodyAccJerkMean),gravityMean)',
 'angle(tBodyAccMean,gravity)',
 'angle(tBodyGyroJerkMean,gravityMean)',
 'angle(tBodyGyroMean,gravityMean)',
 'date_time',
 'fBodyAcc',
 'fBodyAccJerk',
 'fBodyAccMag',
 'fBodyBodyAccJerkMag',
 'fBodyBodyGyroJerkMag',
 'fBodyBodyGyroMag',
 'fBodyGyro',
 'tBodyAcc',
 'tBodyAccJerk',
 'tBodyAccJerkMag',
 'tBodyAccMag',
 'tBodyGyro',
 'tBodyGyroJerk',
 'tBodyGyroJerkMag',
 'tBodyGyroMag',
 'tGravityAcc',
 'tGravityAccMag'}

In [9]:
data['Activity'].value_counts()

Activity
LAYING                16762
WALKING               16728
WALKING_UPSTAIRS      16675
STANDING              16645
WALKING_DOWNSTAIRS    16627
SITTING               16563
Name: count, dtype: int64

In [10]:
data.describe().T

,count,mean,min,25%,50%,75%,max,std
tBodyAcc-mean()-X,100000.0,0.275078,-0.986314,0.262825,0.277178,0.288289,0.965808,0.060068
tBodyAcc-mean()-Y,100000.0,-0.017384,-0.89446,-0.024411,-0.017155,-0.011201,0.87536,0.029089
tBodyAcc-mean()-Z,100000.0,-0.108674,-0.936031,-0.120134,-0.108572,-0.098326,0.999455,0.044118
tBodyAcc-std()-X,100000.0,-0.564353,-1.0,-0.991627,-0.620961,-0.211144,1.0,0.450584
tBodyAcc-std()-Y,100000.0,-0.467669,-0.999629,-0.974517,-0.382965,-0.028442,0.999009,0.503729
...,...,...,...,...,...,...,...,...
"angle(tBodyGyroJerkMean,gravityMean)",100000.0,-0.013572,-1.0,-0.339358,-0.012546,0.30096,0.988742,0.426647
"angle(X,gravityMean)",100000.0,-0.523201,-0.999923,-0.816508,-0.724015,-0.554554,0.997924,0.487325
"angle(Y,gravityMean)",100000.0,0.079965,-0.997831,0.029963,0.18861,0.253884,0.999779,0.293338
"angle(Z,gravityMean)",100000.0,-0.041572,-0.99383,-0.103348,0.002657,0.109922,0.999776,0.256959


In [11]:
le = LabelEncoder()
data['Activity'] = le.fit_transform(data['Activity'])

In [12]:
data['Activity'].value_counts()

Activity
0    16762
3    16728
5    16675
2    16645
4    16627
1    16563
Name: count, dtype: int64

### Feature engineering

In [13]:
from sklearn.model_selection import train_test_split
X = data.drop(['date_time','Activity'] , axis = 1)
Y = data['Activity']

In [14]:
def get_top_k_features(X, Y, k):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X, Y)
    feature_df = pd.DataFrame(data=(X.columns, clf.feature_importances_)).T.sort_values(by=1, ascending=False)
    cols = feature_df.head(k)[0].values
    return cols

In [15]:
top_k_features = get_top_k_features(X, Y, k=10)


In [16]:
top_k_features


array(['tGravityAcc-min()-X', 'tGravityAcc-energy()-X',
       'angle(X,gravityMean)', 'tGravityAcc-min()-Y',
       'tGravityAcc-mean()-X', 'tGravityAcc-max()-Y',
       'tGravityAcc-max()-X', 'angle(Y,gravityMean)',
       'tGravityAcc-mean()-Y', 'tBodyAcc-max()-X'], dtype=object)

## 3. Data preparation

* Split the data into train and test data sets. 
* Regardless of methods used to split the data, compare the ratio of classes in both the train and test splits.


In [17]:
X = X[top_k_features]

In [18]:
x_train , x_test , y_train , y_test = train_test_split(X, Y)


In [19]:
x_train.shape , y_train.shape


((75000, 10), (75000,))

In [20]:
x_test.shape , y_test.shape


((25000, 10), (25000,))

## Standardise data

In [21]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)


## 4. Model Training

* Fit different models and compare the result 
1. Logistic regression
2. Decision Tree Classifier
3. Random Forest Classifier 
4. Adaptive Boosting Classifier

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [23]:
model_result = {}


In [24]:
lr =  LogisticRegression()
lr.fit(x_train_std, y_train)
train_accuracy = round(lr.score(x_train_std, y_train)*100,2)
test_accuracy = round(lr.score(x_test_std, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['Logistic_Regression'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }


Training Accuracy 86.73
Test Accuracy 87.02


/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/code/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
dt =  DecisionTreeClassifier()
dt.fit(x_train, y_train)
train_accuracy = round(dt.score(x_train, y_train)*100,2)
test_accuracy = round(dt.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['Decision_Tree'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }


Training Accuracy 100.0
Test Accuracy 96.88


In [26]:
rfc =  RandomForestClassifier(n_estimators=20)
rfc.fit(x_train, y_train)
train_accuracy = round(rfc.score(x_train, y_train)*100,2)
test_accuracy = round(rfc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['RandomForest'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }


Training Accuracy 99.97
Test Accuracy 98.24


In [27]:
gbc =  GradientBoostingClassifier(n_estimators=20)
gbc.fit(x_train, y_train)
train_accuracy = round(gbc.score(x_train, y_train)*100,2)
test_accuracy = round(gbc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['GradientBoosting'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }

Training Accuracy 87.88
Test Accuracy 87.68


## 5. Model comparison

In [29]:
pd.DataFrame(model_result).T

,train_accuracy,test_accuracy
Logistic_Regression,86.73,87.02
Decision_Tree,100.00,96.88
RandomForest,99.97,98.24
GradientBoosting,87.88,87.68


### 6. Final Model

In [30]:
importances = rfc.feature_importances_


In [31]:
## Top 10 Features contributing to Model 
feature_df = pd.DataFrame( x_train.columns ,  columns = ['variables'])
feature_df['importance'] = importances
feature_df.sort_values( by = 'importance' , ascending = False).head(10)


,variables,importance
9,tBodyAcc-max()-X,0.322295
0,tGravityAcc-min()-X,0.135963
2,"angle(X,gravityMean)",0.117739
7,"angle(Y,gravityMean)",0.080493
3,tGravityAcc-min()-Y,0.067706
5,tGravityAcc-max()-Y,0.064143
6,tGravityAcc-max()-X,0.063113
1,tGravityAcc-energy()-X,0.057226
8,tGravityAcc-mean()-Y,0.053984
4,tGravityAcc-mean()-X,0.037338


In [33]:
y_pred = rfc.predict(x_test)

## 7. Model Evaluation

For each model, calculate the following error metrics: 

* accuracy
* precision
* recall
* fscore
* confusion matrix

Decide how to combine the multi-class metrics into a single value for each model.

In [34]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report


In [35]:
confusion_matrix(y_pred, y_test)

array([[4249,    0,    0,    0,    0,    0],
       [   0, 4051,  174,    0,    0,    0],
       [   0,  127, 4007,    0,    0,    0],
       [   0,    0,    0, 4156,   13,   33],
       [   0,    0,    0,   23, 4039,   21],
       [   0,    0,    0,   33,   16, 4058]])

In [36]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4249
           1       0.97      0.96      0.96      4225
           2       0.96      0.97      0.96      4134
           3       0.99      0.99      0.99      4202
           4       0.99      0.99      0.99      4083
           5       0.99      0.99      0.99      4107

    accuracy                           0.98     25000
   macro avg       0.98      0.98      0.98     25000
weighted avg       0.98      0.98      0.98     25000



## 8. Model registration


In [37]:
## Change Drive path to your Folder
os.mkdir('model_weights/')
os.mkdir('model_features/')

In [38]:
## Let's save model weights
import joblib
# save
joblib.dump(rfc, "./model_weights/my_random_forest.joblib")

['./model_weights/my_random_forest.joblib']

In [39]:
## Save Final columns
final_columns = np.array(x_train.columns) 
joblib.dump(final_columns, "./model_features/train_features.joblib")

['./model_features/train_features.joblib']

## 9. Model Prediction

In [40]:
## load Test Data set 
test_data = pd.read_csv('/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/Human_Activity_Recognition_Using_Smartphones_TestData.csv')

In [41]:
test_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [42]:
## Load Features and model weight
train_features = joblib.load("./model_features/train_features.joblib")

model = joblib.load("./model_weights/my_random_forest.joblib")


In [43]:
test_data_features = test_data[train_features]

In [44]:
test_data_features.fillna(0,inplace = True)

/var/folders/h7/7cnm33m55zqb3ckcp7485gfw0000gn/T/ipykernel_98123/3385524260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_features.fillna(0,inplace = True)


In [45]:
y_prediction = model.predict(test_data_features)
y_prediction_label = le.inverse_transform(y_prediction)
test_data['Prediction_label'] = y_prediction_label

In [46]:
test_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Prediction_label
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING


In [47]:
test_data['Prediction_label'].value_counts()

Prediction_label
LAYING                1944
STANDING              1868
SITTING               1814
WALKING               1708
WALKING_UPSTAIRS      1556
WALKING_DOWNSTAIRS    1409
Name: count, dtype: int64

## End